주피터를 실행하자마자 에러가 떴는데, pip를 새로 설치하면서 jupyter가 pip에 없어서 발생하는 에러였다!  
당황하지 말고 pip install jupyter로 해결해주자.

## 0. Import Modules

In [2]:
# 진행률을 표기하기 위한 tqdm
from tqdm import tqdm

# DataFrame분석을 위한 pandas
import pandas as pd

# validation을 위한 전처리 train_test_split
from sklearn.model_selection import train_test_split

# 분석 및 평가를 위한 sklearn.metrics
from sklearn.metrics import confusion_matrix, accuracy_score

# 시각화 모듈을 위한 matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn

## 1. Read Datasets

In [16]:
# dataset 읽기
# 데이터가 너무 큼...
## 그래서 아래 10분할 해서 train_df-_sample만 사용하기로 했다
train_df_sample = pd.read_csv("./train_df_sample.csv")

In [17]:
# 10%만 사용하니까 그나마 좀 괜찮다.
len(train_df_sample)

2860539

In [4]:
# 가장 무난한 머신러닝 및 scikit learn에 있는 모듈 => Boosting, 그 중에서도 GBM, lightGBM 사용해보자.

## 2. PreProcessing

- EDA
  - null check (isna, fillna)
  - data type (info)
  - outlier check(describe, boxplot)
  - feature selection (~까지 가면 좋겠으나, 지금은 vanilla로 진행) 



```python

"""  
Dataset Info.  
 
train.csv [파일]   
시간 순으로 나열된 7일 동안의 웹 광고 클릭 로그  
ID: train 데이터 샘플 고유 ID  
Click: 예측 목표인 클릭 여부  
0: 클릭하지 않음, 1: 클릭  
F01 ~ F39 : 각 클릭 로그와 연관된 Feature  
개인정보 보호를 위해 상세 정보는 비식별 처리됨  

"""  
```

In [18]:
train_df_sample.head()

,Unnamed: 0,ID,Click,F01,F02,F03,F04,F05,F06,F07,...,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39
0,27047686,TRAIN_27047686,0,CYOAMVC,DJDKEYH,NaN,NaN,XCAJWBW,13,VNXTVLH,...,TFZIBRI,GTISJWW,9787.0,0.0,NPEGYAH,IRUDRFB,15.0,JSOMQYE,0.0,KOFRDGL
1,7223697,TRAIN_07223697,0,JCDXFYU,PILDDJU,IAGJDOH,1.0,LFPUEOV,68,MHBXSWB,...,REPSWLB,KHZNEZF,15063.0,0.0,QMOULXS,IRUDRFB,NaN,XBKBHCW,0.0,LNUQAZZ
2,21459435,TRAIN_21459435,0,VNOHLIR,PKLDGLX,IAGJDOH,5.0,EVTUBMN,6,JNIVDXP,...,SLXYBBG,GTISJWW,601.0,0.0,SHMKPOR,IRUDRFB,6.0,FXWZZCX,0.0,XHYNPHU
3,6264302,TRAIN_06264302,0,UCFAVXY,DJDKEYH,NaN,3.0,WNMKDBA,1,YCWUGFD,...,MFPUCBU,ENBEWZP,4210.0,0.0,DVESEGJ,IRUDRFB,3.0,VDQUXYS,0.0,JDVFUQP
4,22182054,TRAIN_22182054,0,JCDXFYU,PILDDJU,IAGJDOH,11.0,LFPUEOV,81,PIFJCMX,...,NZGEZLW,KHZNEZF,1.0,39.0,QMOULXS,IRUDRFB,0.0,AQGCGGG,1.0,QHKAWMA


In [19]:
train_df_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2860539 entries, 0 to 2860538
Data columns (total 42 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   ID          object 
 2   Click       int64  
 3   F01         object 
 4   F02         object 
 5   F03         object 
 6   F04         float64
 7   F05         object 
 8   F06         int64  
 9   F07         object 
 10  F08         object 
 11  F09         object 
 12  F10         object 
 13  F11         float64
 14  F12         object 
 15  F13         object 
 16  F14         int64  
 17  F15         object 
 18  F16         object 
 19  F17         object 
 20  F18         float64
 21  F19         float64
 22  F20         object 
 23  F21         object 
 24  F22         object 
 25  F23         object 
 26  F24         float64
 27  F25         object 
 28  F26         object 
 29  F27         float64
 30  F28         object 
 31  F29         float64
 32  F30         object 
 33  F31    

In [20]:
train_df_sample.columns

Index(['Unnamed: 0', 'ID', 'Click', 'F01', 'F02', 'F03', 'F04', 'F05', 'F06',
       'F07', 'F08', 'F09', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16',
       'F17', 'F18', 'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F26',
       'F27', 'F28', 'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36',
       'F37', 'F38', 'F39'],
      dtype='object')

In [21]:
x_col = train_df_sample.columns[3:]
print(x_col)

Index(['F01', 'F02', 'F03', 'F04', 'F05', 'F06', 'F07', 'F08', 'F09', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28', 'F29', 'F30',
       'F31', 'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F38', 'F39'],
      dtype='object')


In [22]:
train_y = train_df_sample['Click']
train_x = train_df_sample[x_col]

print(train_x.shape)

(2860539, 39)


In [23]:
# 불필요한 컬럼 드랍.
if 'ID' in train_df_sample.columns:
    train_df_sample.drop(['Unnamed: 0', 'ID'], axis=1, inplace=True)

In [24]:
train_df_sample.columns

Index(['Click', 'F01', 'F02', 'F03', 'F04', 'F05', 'F06', 'F07', 'F08', 'F09',
       'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19',
       'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28', 'F29',
       'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F38', 'F39'],
      dtype='object')

#### different missing value filling method by dtypes

- object type의 경우 결측값 처리 방법을 여러 가지로 가능하다.
  - 위 아래의 값으로 채우기
  - 최빈값으로 채우기
  - 그냥 날려버리기.
  
- numeric type의 경우, 아래와 같은 방법들을 사용할 수 있다.
  - median(중앙값)으로 채우기
  - 0으로 채우기
  
> 그리고 지금 생각나는건...  
> cosine sim을 사용해서 유사도를 구할 수는 없을까?  

`하지만 지금은 Vanilla로 먼저 진행해본다.` (0으로 채우고, nan으로 채우기)



In [25]:
train_object_col = train_df_sample.select_dtypes(['object'])
train_numeric_col = train_df_sample.select_dtypes(exclude=['object'])

print(train_object_col.columns)
print('----'*10) 
print(train_numeric_col.columns)

Index(['F01', 'F02', 'F03', 'F05', 'F07', 'F08', 'F09', 'F10', 'F12', 'F13',
       'F15', 'F16', 'F17', 'F20', 'F21', 'F22', 'F23', 'F25', 'F26', 'F28',
       'F30', 'F31', 'F34', 'F35', 'F37', 'F39'],
      dtype='object')
----------------------------------------
Index(['Click', 'F04', 'F06', 'F11', 'F14', 'F18', 'F19', 'F24', 'F27', 'F29',
       'F32', 'F33', 'F36', 'F38'],
      dtype='object')


In [26]:
train_numeric_col.describe()

,Click,F04,F06,F11,F14,F18,F19,F24,F27,F29,F32,F33,F36,F38
count,2.860539e+06,2.286398e+06,2.860539e+06,2.564728e+06,2.860539e+06,2.127584e+06,2.601387e+06,1.962540e+06,1.754219e+06,1.754219e+06,2.835559e+06,2.601387e+06,2.127584e+06,2.780468e+06
mean,1.943284e-01,3.121925e+01,1.175535e+02,4.063136e+02,9.613142e+00,6.838256e+00,3.433157e-01,1.139983e+02,2.538591e+01,4.529029e+00,1.943406e+04,2.021605e+00,8.156234e+00,1.854519e-01
std,3.956829e-01,4.534102e+02,4.039554e+02,6.634069e+02,1.469758e+01,8.967519e+00,6.112556e-01,5.882291e+02,8.971299e+01,7.833788e+00,7.202122e+04,3.276068e+01,1.873038e+01,1.999053e+00
min,0.000000e+00,1.000000e+00,-1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,3.000000e+00,1.000000e+00,5.700000e+01,1.000000e+00,2.000000e+00,0.000000e+00,7.000000e+00,2.000000e+00,1.000000e+00,3.540000e+02,0.000000e+00,2.000000e+00,0.000000e+00
50%,0.000000e+00,7.000000e+00,5.000000e+00,1.830000e+02,5.000000e+00,4.000000e+00,0.000000e+00,2.700000e+01,6.000000e+00,2.000000e+00,2.711000e+03,0.000000e+00,4.000000e+00,0.000000e+00
75%,0.000000e+00,2.100000e+01,5.000000e+01,4.850000e+02,1.400000e+01,9.000000e+00,1.000000e+00,8.200000e+01,1.800000e+01,4.000000e+00,1.022100e+04,1.000000e+00,1.000000e+01,0.000000e+00
max,1.000000e+00,6.553500e+04,2.687100e+04,8.000000e+03,7.382000e+03,9.270000e+02,1.000000e+01,3.220220e+05,7.546000e+03,2.200000e+02,2.626120e+06,5.132400e+04,1.211200e+04,6.260000e+02


In [31]:
train_numeric_col.fillna(0, inplace=True)

In [32]:
train_object_col.fillna('NAN', inplace=True)

In [33]:
train_numeric_col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2860539 entries, 0 to 2860538
Data columns (total 14 columns):
 #   Column  Dtype  
---  ------  -----  
 0   Click   int64  
 1   F04     float64
 2   F06     int64  
 3   F11     float64
 4   F14     int64  
 5   F18     float64
 6   F19     float64
 7   F24     float64
 8   F27     float64
 9   F29     float64
 10  F32     float64
 11  F33     float64
 12  F36     float64
 13  F38     float64
dtypes: float64(11), int64(3)
memory usage: 305.5 MB


In [34]:
train_numeric_col = train_numeric_col.astype('int64')
train_numeric_col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2860539 entries, 0 to 2860538
Data columns (total 14 columns):
 #   Column  Dtype
---  ------  -----
 0   Click   int64
 1   F04     int64
 2   F06     int64
 3   F11     int64
 4   F14     int64
 5   F18     int64
 6   F19     int64
 7   F24     int64
 8   F27     int64
 9   F29     int64
 10  F32     int64
 11  F33     int64
 12  F36     int64
 13  F38     int64
dtypes: int64(14)
memory usage: 305.5 MB


In [35]:
train_object_col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2860539 entries, 0 to 2860538
Data columns (total 26 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   F01     object
 1   F02     object
 2   F03     object
 3   F05     object
 4   F07     object
 5   F08     object
 6   F09     object
 7   F10     object
 8   F12     object
 9   F13     object
 10  F15     object
 11  F16     object
 12  F17     object
 13  F20     object
 14  F21     object
 15  F22     object
 16  F23     object
 17  F25     object
 18  F26     object
 19  F28     object
 20  F30     object
 21  F31     object
 22  F34     object
 23  F35     object
 24  F37     object
 25  F39     object
dtypes: object(26)
memory usage: 567.4+ MB


In [36]:
train_object_col = train_object_col.astype('category')
train_object_col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2860539 entries, 0 to 2860538
Data columns (total 26 columns):
 #   Column  Dtype   
---  ------  -----   
 0   F01     category
 1   F02     category
 2   F03     category
 3   F05     category
 4   F07     category
 5   F08     category
 6   F09     category
 7   F10     category
 8   F12     category
 9   F13     category
 10  F15     category
 11  F16     category
 12  F17     category
 13  F20     category
 14  F21     category
 15  F22     category
 16  F23     category
 17  F25     category
 18  F26     category
 19  F28     category
 20  F30     category
 21  F31     category
 22  F34     category
 23  F35     category
 24  F37     category
 25  F39     category
dtypes: category(26)
memory usage: 268.6 MB


In [40]:
# lightgbm 설치
# !pip install lightgbm

In [41]:
import lightgbm as lgb

## 3. Modelling

가장 무난하면서도 성능이 좋은 light gbm을 사용해보자.

In [42]:
# 1) train_df에 전처리 한 거 합쳐주기.

num_cols = train_numeric_col.columns
cat_cols = train_object_col.columns

for col in num_cols:
    train_df_sample[col] = train_numeric_col[col]

for col in cat_cols:
    train_df_sample[col] = train_object_col[col]

train_df_sample.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2860539 entries, 0 to 2860538
Data columns (total 40 columns):
 #   Column  Dtype   
---  ------  -----   
 0   Click   int64   
 1   F01     category
 2   F02     category
 3   F03     category
 4   F04     int64   
 5   F05     category
 6   F06     int64   
 7   F07     category
 8   F08     category
 9   F09     category
 10  F10     category
 11  F11     int64   
 12  F12     category
 13  F13     category
 14  F14     int64   
 15  F15     category
 16  F16     category
 17  F17     category
 18  F18     int64   
 19  F19     int64   
 20  F20     category
 21  F21     category
 22  F22     category
 23  F23     category
 24  F24     int64   
 25  F25     category
 26  F26     category
 27  F27     int64   
 28  F28     category
 29  F29     int64   
 30  F30     category
 31  F31     category
 32  F32     int64   
 33  F33     int64   
 34  F34     category
 35  F35     category
 36  F36     int64   
 37  F37     category
 38  F3

In [47]:
train_y = train_df_sample['Click']
train_x = train_df_sample.drop(['Click'], axis=1)

train_x.shape

(2860539, 39)

### Training

In [49]:
model = lgb.LGBMClassifier(objective='binary')
model.fit(train_x, train_y, eval_metric='AUC')

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 555884, number of negative: 2304655
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.319565 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 74681
[LightGBM] [Info] Number of data points in the train set: 2860539, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.194328 -> initscore=-1.422127
[LightGBM] [Info] Start training from score -1.422127


LGBMClassifier(objective='binary')

In [71]:
## sample로 사용할 거라면, 앞으로는 꼭 함수화하자.
# preprocess

def load_data(path):    
    df = pd.read_csv(path)
    
    df.drop('ID', axis=1, inplace=True)
    obj_cols_df = df.select_dtypes(include='object')
    num_cols_df = df.select_dtypes(exclude='object')
     
    # fill missing values
    num_cols_df.fillna(0, inplace=True)
    obj_cols_df.fillna('NAN', inplace=True)

    # replace origin to cleansed column
    num_cols = num_cols_df.columns
    obj_cols = obj_cols_df.columns
    df[num_cols] = num_cols_df[num_cols].astype('int64')
    df[obj_cols] = obj_cols_df[obj_cols].astype('category')

    return df

In [72]:
test_df = load_data("./datasets/test.csv")

In [73]:
pred = model.predict_proba(test_df)

In [74]:
sample_submission = pd.read_csv('./datasets/sample_submission.csv')
sample_submission

,ID,Click
0,TEST_0000000,0
1,TEST_0000001,0
2,TEST_0000002,0
3,TEST_0000003,0
4,TEST_0000004,0
...,...,...
4538536,TEST_4538536,0
4538537,TEST_4538537,0
4538538,TEST_4538538,0
4538539,TEST_4538539,0


In [75]:
sample_submission['Click'] = pred[:,1]
sample_submission

,ID,Click
0,TEST_0000000,0.200314
1,TEST_0000001,0.168957
2,TEST_0000002,0.182148
3,TEST_0000003,0.233223
4,TEST_0000004,0.297747
...,...,...
4538536,TEST_4538536,0.129919
4538537,TEST_4538537,0.229020
4538538,TEST_4538538,0.107711
4538539,TEST_4538539,0.247740


In [76]:
sample_submission.to_csv('lgbm.csv', index=False)